DATAFRAME->its like a SQL table

In [ ]:
import pandas as pd
import numpy as np
from unidecode import unidecode


df = pd.DataFrame([[1,2,3],
                   [4,5,6],
                   [7,8,9]] , columns=['A','B','C'] , index=['x','y','z'])
df['A'].unique() #UNIQUE ELEMENTS OF COLUMN 'A'


IMPORTING FILES INTO DATAFRAMES

In [ ]:
bios2 = pd.read_csv('https://raw.githubusercontent.com/KeithGalli/complete-pandas-tutorial/refs/heads/master/data/bios.csv')
coffee = pd.read_csv('https://raw.githubusercontent.com/KeithGalli/complete-pandas-tutorial/refs/heads/master/warmup-data/coffee.csv')
bios = pd.read_csv('https://raw.githubusercontent.com/KeithGalli/complete-pandas-tutorial/refs/heads/master/data/bios.csv')

MODIFYING COLUMN NAMES

In [ ]:
coffee.rename(columns={'Day':'day' , 'units_sols': 'units sold' , 'Coffee Type':'coffee type'} , inplace=True) #or coffee.columns = coffee.columns.set.lower()
coffee.columns = coffee.columns.str.replace('_',' ')
coffee

ACCESSING DATA IN PANDAS

In [ ]:
coffee.sample(10)

coffee.loc[1] #loc -> access by row number
coffee.loc[[0,1,5]]

#SLICING IN LOC ans ILOC (this is also rows,column ... but columns have to be named)
coffee.loc[5:8,['day','units sold']] #in iloc this will be [5:9(in iloc the end point is exclusive) , [0,2]]

#by column name
coffee['units sold']
e
#sorting
coffee.sort_values('units sold')
coffee.sort_values('units sold' , ascending=False)
coffee.sort_values(['units sold','coffee type'] , ascending=[1,0]) #sort by units sold (ascending=true) first, in case they are same then look at coffee type(ascending=false)

#NOTE: INPLACE MEANS MODIFYING THE EXISTING DF, ELSE A NEW DF IS CREATED
#NOTE: SORTING DOES NOT REARRANGE THE INDEXES, TO ACHIEVE THAT:-
coffee2 = coffee.sort_values(by='day').reset_index(drop=True) #coffee 2 is the modified DF , if you wanna modify the original DF do coffee=
#NOTE: SIMPLY WRITING coffee.sort_values(by='day').reset_index(drop=True) WILL DO NOTHING

FILTERING DATA

In [ ]:
import unicodedata
bios.loc[(bios['height_cm']>200) & (bios['weight_kg']>100)]
bios.loc[(bios['height_cm']>200) & (bios['weight_kg']>100) , ['name','height_cm','weight_kg']] #cause in the end its .loc[row,column]

bios.loc[ (bios['height_cm']>200) & (bios['weight_kg']>100) & (bios['born_country'] == 'USA') , ['name','height_cm','weight_kg','born_country']] #multiple conditions

bios[bios['name'].str.contains('shaq', case = False) ] #case sensitive by default


#SMALL CLEANING EXAMPLE ->ARRANGING NAMES COLUMN IN ALPHABETIC ORDER


In [ ]:
from unidecode import unidecode
bios2 = (bios2.sort_values(
    by='name',
    key=lambda x: x.apply(unidecode).str.lower() 
).reset_index(drop=True) #this line rearranges the rows as well (mentioned above)
)

What problem this solves (context)
Your name column contains:-
  accented letters (José)
  Cyrillic look-alikes (Аlex)
  mixed casing (alex, Alex)
Normal sorting uses Unicode order, which gives weird results.

key=... → transform before sorting
key tells Pandas:
“Before comparing values, transform them using this function.”
Important:
The transformation is temporary :-
  is used only to compare values
  is NOT stored
  does NOT replace the column  
  exists only during sorting

x.apply(unidecode) -> treats è = e and à = a etc
.str.lower() ->treats alex, Alex, ALEX same

#ADDING/REMOVING COLUMNS

In [ ]:
coffee['price'] = 4.99 #add column with default value
coffee['new price'] = np.where(coffee['Coffee Type'].str.lower() == 'espresso' , 5.99 , 4.99) #conditional assignment

#METHOD2
conditions = [
    coffee['Coffee Type'].str.lower() == 'espresso',
    coffee['Coffee Type'].str.lower() == 'latte'
]
prices = [5.99 , 6.99]
coffee['new price'] = np.select(conditions , prices , default=4.99)

coffee.drop(columns=['price'], inplace=True) #drop column

#METHOD3 = conditional adressing without NUMPY 
coffee.loc[coffee['Coffee Type'] == 'Espresso' , 'price'] = 5.99
coffee.loc[coffee['Coffee Type'] == 'Latte' , 'price'] = 6.99
coffee.drop(columns=['new price'], inplace=True)
coffee

if you wanna make a copy of a df and modify it without changing the original df, this is the method:-

In [ ]:
#coffee_new = coffee.copy() #coffee_new = coffee is wrong as coffee_new will point to same memory location as coffee
coffee_new = coffee
coffee_new['price'] = 4 #this will also add price column to coffee df as both point to same memory location

coffee_new['revenue'] = coffee_new['Units Sold'] * coffee_new['price'] #adding new column based on other columns


In [ ]:
#OLYMPICS DATA ->
bios_new = bios.copy()
bios_new['height_category'] = bios_new['height_cm'].apply(lambda x: 'short' if x<160 else 'average' if x<185 else 'unknown' if pd.isna(x) else 'tall')

bios_new['normalised_name'] = bios_new['name'].str.lower().apply(unidecode)
bios_new = bios_new.sort_values(by='normalised_name').reset_index(drop=True)
 
bios_new = bios_new.drop(columns=['normalised_name']) #we dont need the normalised_name column now
bios_new 

HANDLING NaN's

In [ ]:
#coffee.loc[2:3,'Units Sold'] = np.nan
coffee
#coffee.fillna(coffee['Units Sold'].mean() , inplace=True) #filling missing values with mean of the column
coffee
coffee.fillna(coffee['Units Sold'].interpolate() , inplace=True) #looks at neighbours and fills acordingly
coffee
#coffee.dropna(inplace=True) #drop rows with missing values
coffee.dropna(subset='Units Sold',inplace=True) #drop rows only if 'Units Sold' is missing



AGGREGATING DATA

In [ ]:
bios['NOC'].value_counts()
bios['NOC'].value_counts(normalize=True) #percentage
bios['height_cm'].value_counts(bins=3) #creates interval acc. to number of bins(intervals) that we provide, works only for numerical data
bios[bios['born_country'] == 'USA']['height_cm'].value_counts(bins=3) #multiple data
bios[bios['born_country'] == 'USA']['born_region'].value_counts() #example 2

In [ ]:
coffee